In [1]:
import numpy as np 
from scipy.integrate import quad

In [ ]:
class Schulten_Wolynes:

    # Initialises the class 
    # parameters
    #   applied_magnetic_field  : the strength of the applied magnetic field
    #   gyromagnetic_ratio  : gyromagnetic ratio of the electron in radical i 
    #   nuclei_number   : number of nuclei present
    #   total_time      : total_time to simulate the tensor over
    #   time_steps      : the number of points to simulate the tensor over
    #   isotropic_coupling_constant : hyperfine coupling constants between nuclear spin and electron spin


    def __init__(self,applied_magnetic_field_strength,gyromagnetic_ratio,nuclei_number,total_time,time_steps,isotropic_coupling_constant):
          self.applied_magnetic_field_strength = applied_magnetic_field_strength
          self.gyromagnetic_ratio = gyromagnetic_ratio
          self.nuclei_number = nuclei_number
          self.total_time = total_time
          self.time_steps = time_steps
          self.isotropic_coupling_constant = isotropic_coupling_constant

    # Calculate the specificed electron spin correlation tensor for the total time 

    def calculate_electron_tensor(self,tensor): 
         electron_spin_correlation_tensor = []
         t_star_array = []
         func_w_star_t_star_array = []
         tau = self.__calculate_tau__()
         w_star = self.__calculate_w_star__(tau)
         time_increment = self.total_time / self.time_steps
         
         for i in range(self.time_steps):
              
                current_time = i * time_increment
                t_star = current_time / tau
                func_w_star_t_star = self.__calculate_func_numerically__(t_star,w_star) # very inefficient 
              
                tensor_function = getattr(self, tensor)
            
                t_star_array.append(t_star)
                func_w_star_t_star_array.append(func_w_star_t_star)
                electron_spin_correlation_tensor.append(tensor_function(w_star, t_star, func_w_star_t_star))
         
         return np.real(electron_spin_correlation_tensor)
        
    
    def __integrand__(self,s, w):
        return np.exp(-s**2) * np.sin(w * s)
         

    def __calculate_func_numerically__(self,t_star,w_star):
         func_w_star_t_star_value, error = quad(self.__integrand__, 0 ,t_star, args=(w_star))
         return func_w_star_t_star_value 


    def __calculate_tau__(self):
        tau = np.sqrt(6 / (np.sum(np.square(self.isotropic_coupling_constant[0:self.nuclei_number]))*0.5*(0.5 + 1)))
        return  tau


    def __calculate_w_star__(self,tau):
        w_star = -self.gyromagnetic_ratio * self.applied_magnetic_field_strength * tau
        return  abs(w_star)
              
    # Closed form expression of electron spin correlation tensors 

    def __R_xx__(self, w_star, t_star, func_w_star_t_star): 
        return (w_star * (2 + np.exp(-t_star**2) * ((w_star**2 - 2) * np.cos(w_star * t_star) - 2 * w_star * t_star * np.sin(w_star * t_star))) - 4 * func_w_star_t_star) / (2 * w_star**3)
        

    def __R_xy__(self,w_star,t_star, func_w_star_t_star):
        return np.exp(-t_star**2)*(2*w_star*t_star*np.cos(w_star * t_star) + (w_star**2 -2)*np.sin(w_star * t_star)) / (2* w_star**2)


    def __R_zz__(self, w_star, t_star, func_w_star_t_star):
        return (w_star * ((w_star** 2 ) + (4 * np.exp(-t_star ** 2) * np.cos(w_star * t_star)) -4) + ( 8 * func_w_star_t_star)) / (2 * (w_star ** 3))